In [1]:
import numpy as np
import pandas as pd
import time

from functools import partial
from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None

In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.08)

In [3]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


556.1680109500885

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(2792.237649427154, 0.01396118824713577, 174)

In [5]:
output['history']

,impressions,clicks,lifetime,p
132,19843.0,1332.0,10870.812904,0.064972
162,154.0,2.0,18594.827945,0.017514
163,86.0,0.0,5153.010937,0.000849
164,68.0,2.0,5092.571727,0.041281
166,48.0,0.0,5340.552070,0.007253
167,33.0,0.0,1474.181162,0.033849
168,23.0,0.0,4900.260295,0.027273
169,14.0,0.0,5007.022458,0.030857
170,18.0,0.0,8920.324215,0.034653
171,14.0,0.0,1080.025985,0.012549


# UCB policy exploration

In [6]:
def ucb1(history: pd.DataFrame) -> int:
    N = history.shape[0]
    ctr_quantile = history['clicks']/(history['impressions']+10) + \
    np.sqrt(
        2*np.log(N) / (history['impressions'] + 10)
    )
    n = np.argmax(ctr_quantile)
    return history.index[n]

In [7]:
policy = ucb1

In [8]:
%%time

output = simulation(policy, n=200000)

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
CPU times: user 10min 39s, sys: 1.12 s, total: 10min 40s
Wall time: 10min 38s


In [9]:
# UCB1 regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(4892.203243302422, 0.024461016216512112, 208)

In [10]:
output['history']

,impressions,clicks,lifetime,p
152,963.0,56.0,3542.628628,0.060567
170,1659.0,120.0,8646.852102,0.074247
193,325.0,7.0,22108.414021,0.014121
200,207.0,2.0,2271.421589,0.013661
201,195.0,1.0,16738.531244,0.006106
202,6780.0,884.0,3972.106269,0.128287
203,174.0,0.0,14699.823825,0.002851
204,569.0,39.0,17806.294214,0.064651
207,362.0,17.0,1155.749592,0.047443


## Tune simple UCB

In [26]:
def ucb_tuned(history: pd.DataFrame, alpha: float) -> int:
    N = history.shape[0]
    ctr_quantile = history['clicks'] / (history['impressions']+1) + \
    alpha*np.sqrt(
        2*np.log(N) / (history['impressions']+1)
    )
    n = np.argmax(ctr_quantile)
    return history.index[n]

In [31]:
%%time

N = 1000*50
for alpha in np.logspace(-3, 3, 7):
    policy = partial(ucb_tuned, alpha=alpha)
    output = simulation(policy, n=N)
    print(f'alpha={alpha}, regret={output["regret"]}')

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=0.001, regret=1906.6663896169532
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=0.01, regret=2043.6414221179327
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=0.1, regret=541.6185061863391
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=1.0, regret=921.4744910556321
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulat

In [38]:
%%time

N = 1000*50
for alpha in np.logspace(-2, 0, 7):
    policy = partial(ucb_tuned, alpha=alpha)
    output = simulation(policy, n=N)
    print(f'alpha={alpha:.2e}, regret={output["regret"]}')

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=1.00e-02, regret=1336.7892078821503
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=2.15e-02, regret=3384.7667746769043
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=4.64e-02, regret=2037.6978473830263
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=1.00e-01, regret=795.441545758774
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions 

In [39]:
%%time

N = 1000*50
for alpha in np.linspace(1e-1, 5e-1, 7):
    policy = partial(ucb_tuned, alpha=alpha)
    output = simulation(policy, n=N)
    print(f'alpha={alpha:.2e}, regret={output["regret"]}')

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=1.00e-01, regret=624.013181329412
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=1.67e-01, regret=311.38398131897577
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=2.33e-01, regret=184.89370389578963
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
alpha=3.00e-01, regret=249.9129904128437
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions h

KeyboardInterrupt: 

In [40]:
%%time

# compute the regret for the long run
policy = partial(ucb_tuned, alpha=2.3e-1)
output = simulation(policy, n=200000)

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
CPU times: user 10min 43s, sys: 2.56 s, total: 10min 46s
Wall time: 25min 43s


In [43]:
# tuned UCB1 regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(609.5911734462886, 0.003047955867231443, 191)

In [44]:
output['history']

,impressions,clicks,lifetime,p
161,1614.0,69.0,1418.246081,0.046422
178,641.0,33.0,12000.196317,0.061000
183,11.0,0.0,4219.745822,0.020941
187,119.0,14.0,1991.923114,0.131363
188,10.0,0.0,8124.162340,0.034105
189,10.0,0.0,952.905392,0.071298
190,10.0,0.0,2705.975604,0.033297


tuned UCB1 regret: 610

e-greedy baseline: 2792

:)